# Project Metadata & Setup

---

## **Project Title:** **Early Breast Cancer Diagnosis using Machine Learning (Terminal-Based Prototype)**

---

### **Objective:**

Develop a lightweight, terminal-driven diagnostic prototype that predicts breast tumor malignancy using the built-in Breast Cancer Wisconsin dataset.
The goal is to simulate how a clinician or technician might use a fast, interpretable tool, without a full GUI or web app.


### **Dataset Description:**

* **Name:** Breast Cancer Wisconsin (Diagnostic)
* **Source:** `sklearn.datasets.load_breast_cancer()`
* **Samples:** 569
* **Features:** 30 numerical (e.g. radius, texture, symmetry)
* **Target:** Binary classification — `malignant (0)` vs. `benign (1)`



### **Stakeholders:**

| Stakeholder          | Interest / Use Case                                              |
| -------------------- | ---------------------------------------------------------------- |
| **Clinicians**       | Fast, interpretable predictions without cloud dependency         |
| **Researchers**      | Baseline model to compare with deep learning or ensemble methods |
| **Medical Startups** | Prototype backend logic for terminal-based tools                 |
| **Educators**        | Teaching ML with meaningful, real-world data                     |


### **Key Notes for Revision:**

* **No GPU required** — uses efficient `RandomForestClassifier`
* **Runs locally** in terminal with zero external dataset download
* **Emphasis on simplicity**: ASCII feedback, clean input prompts
* **Can be modularized later** into API or UI backend
* **We’ll skip test/train splitting** for now — prototype phase only


### **Environment Requirements:**

| Package    | Use                          |
| ---------- | ---------------------------- |
| `sklearn`  | Dataset + ML model           |
| `pandas`   | Tabular data inspection      |
| `colorama` | Terminal coloring (optional) |
| `numpy`    | Data handling                |

---

### Install (if not present):

```bash
pip install pandas scikit-learn colorama
```


### Deliverables:

* Classifier trained on breast cancer dataset
* Real-time terminal prediction system
* ASCII output or basic feedback UI for predicted diagnosis
* Clean, commented code blocks
* Professional markdown formatting with revision comments


## STEP 1: Load and Explore the Breast Cancer Dataset


### **Objective:**

* Load the Breast Cancer Wisconsin dataset directly from `sklearn`
* Convert to a `pandas.DataFrame` for readability
* Summarize the dataset structure and feature information
* Confirm class distribution (very important in medical ML)


### **Background Insight for Stakeholders:**

* The data represent **digitized characteristics of cell nuclei** from fine-needle aspirates of breast masses.
* 30 numerical features are calculated from images (e.g., mean radius, standard error of texture, worst smoothness).
* The target is **binary**:

  * `0 = malignant` (cancerous)
  * `1 = benign` (non-cancerous)


### **Code Block: Load + Inspect Data**

In [12]:
# STEP 1: Load & Inspect Breast Cancer Dataset
from sklearn.datasets import load_breast_cancer
import pandas as pd

# Load dataset from sklearn
cancer = load_breast_cancer()

# Convert to pandas DataFrame
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)

# Add target labels
df['target'] = cancer.target
df['target_name'] = df['target'].map(lambda x: cancer.target_names[x])

# Summary outputs
print("🧬 Dataset Dimensions:", df.shape)
print("🔢 Number of Features:", len(cancer.feature_names))
print("🎯 Target Labels:", list(cancer.target_names))
print("\n📊 Class Distribution:")
print(df['target_name'].value_counts())

# Preview sample records
df.sample(5)

🧬 Dataset Dimensions: (569, 32)
🔢 Number of Features: 30
🎯 Target Labels: [np.str_('malignant'), np.str_('benign')]

📊 Class Distribution:
target_name
benign       357
malignant    212
Name: count, dtype: int64


mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
375        16.17         16.07          106.30      788.5          0.09880   
270        14.29         16.82           90.30      632.6          0.06429   
243        13.75         23.77           88.54      590.0          0.08043   
50         11.76         21.60           74.72      427.9          0.08637   
328        16.27         20.71          106.90      813.7          0.11690   

     mean compactness  mean concavity  mean concave points  mean symmetry  \
375           0.14380         0.06651              0.05397         0.1990   
270           0.02675         0.00725              0.00625         0.1508   
243           0.06807         0.04697              0.02344         0.1773   
50            0.04966         0.01657              0.01115         0.1495   
328           0.13190         0.14780              0.08488         0.1948   

     mean fractal dimension  ...  worst perimeter  worst area  \
375                 0.06572  ...           113.10       861.5   
270                 0.05376  ...            94.44       684.6   
243                 0.05429  ...            98.00       706.0   
50                  0.05888  ...            82.98       516.5   
328                 0.06277  ...           129.80      1121.0   

     worst smoothness  worst compactness  worst concavity  \
375           0.12350            0.25500          0.21140   
270           0.08567            0.05036          0.03866   
243           0.09368            0.14420          0.13590   
50            0.10850            0.08615          0.05523   
328           0.15900            0.29470          0.35970   

     worst concave points  worst symmetry  worst fractal dimension  target  \
375               0.12510          0.3153                  0.08960       1   
270               0.03333          0.2458                  0.06120       1   
243               0.06106          0.2663                  0.06321       1   
50                0.03715          0.2433                  0.06563       1   
328               0.15830          0.3103                  0.08200       0   

     target_name  
375       benign  
270       benign  
243       benign  
50        benign  
328    malignant  

[5 rows x 32 columns]


### Notes for Revision:

* **Shape:** `(569, 32)` → 569 observations, 30 features + 1 label + 1 label name
* **Target imbalance:** More benign than malignant cases → might require stratified validation later
* `target_name` column added for human-readability
###  Example Output (Typical):

🧬 Dataset Dimensions: (569, 32)
🔢 Number of Features: 30
🎯 Target Labels: ['malignant', 'benign']

📊 Class Distribution:
benign       357
malignant    212
Name: target_name, dtype: int64


## STEP 2: Train the Classifier


### Objective:

Train a **Random Forest Classifier** on the full dataset.
This prototype focuses on prediction responsiveness — no test/train split or cross-validation yet.


### Model Choice Justification:

| Model                    | Reason for Selection                                                         |
| ------------------------ | ---------------------------------------------------------------------------- |
| `RandomForestClassifier` | Fast to train, robust to outliers, good with non-linear feature interactions |
| `n_estimators=100`       | Balances accuracy and inference speed                                        |
| `random_state=42`        | Ensures repeatable results                                                   |

This setup fits our terminal-based use case — quick predictions and stable accuracy.


### Code Block: Train the Model




In [13]:
# STEP 2: Train Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

# Prepare feature matrix X and target vector y
X = cancer.data
y = cancer.target

# Initialize classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train on entire dataset (prototype phase)
clf.fit(X, y)

# Evaluate on training set
train_accuracy = clf.score(X, y)
print(f"Training Accuracy: {train_accuracy:.2%}")

Training Accuracy: 100.00%


### Notes for Revision:

* `clf.fit(X, y)` trains on all 569 samples
* Accuracy may look very high (\~99%) — acceptable for a prototype, but must validate later with real test data
* For a full deployment, we’d use `train_test_split()` and stratified validation folds

## New Scenario:

### **“Quick Diagnosis Assistant”**

Instead of inputting all 30 features manually, we:

* **Randomly sample a case** from the dataset
* Display its features to simulate "incoming patient data"
* Ask the user:
  *"Would you like to diagnose this case?"*
* Then we **predict and show results** using the trained model

No typing 30 numbers. Still real. Still sharp. Still interactive.


## STEP 3 (Revised): Sample and Diagnose a Case


### Objective:

* Randomly pull a sample case from the dataset
* Show the 5–7 most meaningful features (not all 30)
* Let the model predict and show the result
* Give option to keep looping or exit

In [15]:
# STEP 3 (REVISED): Simulated Patient Diagnosis
import numpy as np
import random

# Define which features to show (select top 6 visually intuitive ones)
selected_features = [
    'mean radius',
    'mean texture',
    'mean perimeter',
    'mean area',
    'worst concavity',
    'worst symmetry'
]

# Get feature indices
feature_indices = [list(cancer.feature_names).index(f) for f in selected_features]

def sample_and_diagnose(model, X, y, feature_names, n=1):
    while True:
        index = random.randint(0, len(X) - 1)
        sample = X[index]
        label = y[index]
        
        print("\n--- New Patient Case ---")
        for i in feature_indices:
            print(f"{feature_names[i]}: {sample[i]:.2f}")
        
        confirm = input("\nRun diagnosis? (y/n): ").strip().lower()
        if confirm != 'y':
            cont = input("Skip to next case? (y/n): ").strip().lower()
            if cont != 'y':
                print("\nSession ended.")
                break
            else:
                continue

        # Predict
        pred = model.predict(sample.reshape(1, -1))[0]
        label_actual = cancer.target_names[label]
        label_pred = cancer.target_names[pred]
        
        print("\nDiagnosis Prediction:")
        print(f"  → Predicted: {label_pred.upper()}")
        print(f"  → Actual:    {label_actual.upper()}")
        
        cont = input("\nRun another case? (y/n): ").strip().lower()
        if cont != 'y':
            print("\nSession ended.")
            break

# Run the simulation
sample_and_diagnose(clf, cancer.data, cancer.target, cancer.feature_names)


--- New Patient Case ---
mean radius: 9.90
mean texture: 18.06
mean perimeter: 64.60
mean area: 302.40
worst concavity: 0.35
worst symmetry: 0.26



Run diagnosis? (y/n):  y



Diagnosis Prediction:
  → Predicted: BENIGN
  → Actual:    BENIGN



Run another case? (y/n):  y



--- New Patient Case ---
mean radius: 10.75
mean texture: 14.97
mean perimeter: 68.26
mean area: 355.30
worst concavity: 0.10
worst symmetry: 0.23


...


--- New Patient Case ---
mean radius: 12.95
mean texture: 16.02
mean perimeter: 83.14
mean area: 513.70
worst concavity: 0.22
worst symmetry: 0.34



Run diagnosis? (y/n):  y



Diagnosis Prediction:
  → Predicted: BENIGN
  → Actual:    BENIGN



Run another case? (y/n):  



Session ended.


### Code Block: Simulated Case Prediction

### Notes:

* You review a patient case like a **doctor with a clipboard**
* Model handles backend prediction
* You decide if it should run or skip
* Actual label shown for reference, but this could be hidden in real use

## STEP 4: Add Confidence Scores and Model Certainty

### Objective:

Display **how confident** the model is in its prediction, not just the class label.

This is critical in any clinical or decision-support context — we don’t just want a binary label, we want to know **how certain** the model is.


### Tools Used:

| Function              | Purpose                              |
| --------------------- | ------------------------------------ |
| `predict_proba(X)`    | Returns array of class probabilities |
| `np.max()`            | Extracts top confidence score        |
| `colorama` (optional) | Highlights high/low confidence       |


### Code Block: Display Confidence

In [18]:
from sklearn.metrics import accuracy_score
from colorama import Fore, Style

def sample_and_diagnose_with_confidence(model, X, y, feature_names, n=1):
    while True:
        index = random.randint(0, len(X) - 1)
        sample = X[index]
        label = y[index]
        
        print("\n--- New Patient Case ---")
        for i in feature_indices:
            print(f"{feature_names[i]}: {sample[i]:.2f}")
        
        confirm = input("\nRun diagnosis? (y/n): ").strip().lower()
        if confirm != 'y':
            cont = input("Skip to next case? (y/n): ").strip().lower()
            if cont != 'y':
                print("\nSession ended.")
                break
            else:
                continue

        # Predict and get probability
        pred = model.predict(sample.reshape(1, -1))[0]
        prob = model.predict_proba(sample.reshape(1, -1))[0]
        confidence = np.max(prob)
        
        label_actual = cancer.target_names[label]
        label_pred = cancer.target_names[pred]

        print("\nDiagnosis Prediction:")
        if confidence >= 0.90:
            color = Fore.GREEN
        elif confidence >= 0.75:
            color = Fore.YELLOW
        else:
            color = Fore.RED

        print(f"  → Predicted: {label_pred.upper()}")
        print(f"  → Confidence: {color}{confidence:.2%}{Style.RESET_ALL}")
        print(f"  → Actual:    {label_actual.upper()}")

        cont = input("\nRun another case? (y/n): ").strip().lower()
        if cont != 'y':
            print("\nSession ended.")
            break

# Run enhanced simulation
sample_and_diagnose_with_confidence(clf, cancer.data, cancer.target, cancer.feature_names)


--- New Patient Case ---
mean radius: 13.00
mean texture: 20.78
mean perimeter: 83.51
mean area: 519.40
worst concavity: 0.08
worst symmetry: 0.32



Run diagnosis? (y/n):  y



Diagnosis Prediction:
  → Predicted: BENIGN
  → Confidence: 100.00%
  → Actual:    BENIGN



Run another case? (y/n):  y



--- New Patient Case ---
mean radius: 12.58
mean texture: 18.40
mean perimeter: 79.83
mean area: 489.00
worst concavity: 0.01
worst symmetry: 0.25


...


--- New Patient Case ---
mean radius: 14.42
mean texture: 19.77
mean perimeter: 94.48
mean area: 642.50
worst concavity: 0.32
worst symmetry: 0.27



Run diagnosis? (y/n):  y



Diagnosis Prediction:
  → Predicted: MALIGNANT
  → Confidence: 90.00%
  → Actual:    MALIGNANT



Run another case? (y/n):  



Session ended.


### Notes:

* Adds `predict_proba()` to show probability
* Uses color to **visually flag confidence**:

  * Green = strong confidence
  * Yellow = moderate
  * Red = weak/conflicted
* More aligned with how clinicians review uncertainty in diagnostics

**Step 4 complete. Time to close the loop with Step 5 — logging and export.**


## STEP 5: Log Predictions for Audit or Review

### 🎯 Objective:

Record each prediction in a **session log** for review, audit, or retraining.
In medical tools, **traceability** is essential — even in prototypes.

We'll log:

* Sample index
* Selected feature values
* Predicted class
* Actual class
* Model confidence

The log can be printed live or saved to CSV/JSON for analysis.

### Code Block: Logging Predictions


In [21]:
import csv

def sample_and_diagnose_with_logging(model, X, y, feature_names, log_path="diagnosis_log.csv"):
    log_fields = [
        'index',
        'predicted_label',
        'actual_label',
        'confidence',
    ] + [feature_names[i] for i in feature_indices]

    log_data = []

    while True:
        index = random.randint(0, len(X) - 1)
        sample = X[index]
        label = y[index]

        print("\n--- New Patient Case ---")
        for i in feature_indices:
            print(f"{feature_names[i]}: {sample[i]:.2f}")
        
        confirm = input("\nRun diagnosis? (y/n): ").strip().lower()
        if confirm != 'y':
            cont = input("Skip to next case? (y/n): ").strip().lower()
            if cont != 'y':
                print("\nSession ended.")
                break
            else:
                continue

        pred = model.predict(sample.reshape(1, -1))[0]
        prob = model.predict_proba(sample.reshape(1, -1))[0]
        confidence = np.max(prob)

        label_pred = cancer.target_names[pred]
        label_actual = cancer.target_names[label]

        print("\nDiagnosis Prediction:")
        print(f"  → Predicted: {label_pred.upper()}")
        print(f"  → Confidence: {confidence:.2%}")
        print(f"  → Actual:    {label_actual.upper()}")

        # Log entry
        entry = {
            'index': index,
            'predicted_label': label_pred,
            'actual_label': label_actual,
            'confidence': round(confidence, 4)
        }
        for i in feature_indices:
            entry[feature_names[i]] = round(sample[i], 3)
        log_data.append(entry)

        cont = input("\nRun another case? (y/n): ").strip().lower()
        if cont != 'y':
            break

    # Save log at end
    with open(log_path, mode='w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=log_fields)
        writer.writeheader()
        writer.writerows(log_data)

    print(f"\nSession log saved to: {log_path}")

# Run with logging enabled
sample_and_diagnose_with_logging(clf, cancer.data, cancer.target, cancer.feature_names)


--- New Patient Case ---
mean radius: 9.85
mean texture: 15.68
mean perimeter: 63.00
mean area: 293.20
worst concavity: 0.08
worst symmetry: 0.25



Run diagnosis? (y/n):  y



Diagnosis Prediction:
  → Predicted: BENIGN
  → Confidence: 99.00%
  → Actual:    BENIGN



Run another case? (y/n):  y



--- New Patient Case ---
mean radius: 19.59
mean texture: 25.00
mean perimeter: 127.70
mean area: 1191.00
worst concavity: 0.40
worst symmetry: 0.23


...


--- New Patient Case ---
mean radius: 8.20
mean texture: 16.84
mean perimeter: 51.71
mean area: 201.90
worst concavity: 0.07
worst symmetry: 0.31



Run diagnosis? (y/n):  y



Diagnosis Prediction:
  → Predicted: BENIGN
  → Confidence: 100.00%
  → Actual:    BENIGN



Run another case? (y/n):  



Session log saved to: diagnosis_log.csv


### Notes:

* Log is written to `diagnosis_log.csv` in the current directory
* Columns: index, predicted/actual, confidence, plus select features
* Can later be loaded into pandas for analysis or model debugging